In [1]:
# import pandas and numpy

import pandas as pd
import numpy as np


# import sklearn

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# fairlearn metrics

import fairlearn
from fairlearn.metrics import MetricFrame
from fairlearn.metrics import selection_rate, demographic_parity_ratio, demographic_parity_difference
from fairlearn.metrics import false_negative_rate, equalized_odds_ratio, equalized_odds_difference


# fairlearn reductions

from fairlearn.reductions import DemographicParity, EqualizedOdds


# fairlearn postprocessing

from fairlearn.postprocessing import ThresholdOptimizer

In [2]:
# load in preprocessed Coswara dataset and display first 5 rows

data = pd.read_csv("../preprocessed_coswara_data.csv")
data.head()

,Age,Gender,Covid Status,1,2,3,4,5,6,7,...,31,32,33,34,35,36,37,38,39,40
0,young,male,negative,-707.02594,56.426230,13.907639,17.147783,-2.012528,5.152593,15.529266,...,-0.237741,3.750417,10.081942,-9.412180,-3.947216,8.084291,-2.917649,3.018960,-1.533405,4.374303
1,young,male,negative,-427.06598,24.380165,53.189840,-27.081852,-17.267035,22.450130,-9.862848,...,0.229314,-5.317071,7.775287,5.483598,-1.584106,0.640845,-8.218983,8.017080,3.545668,9.140753
2,young,male,negative,-576.46340,189.395810,-47.168960,54.421180,-34.666393,48.699585,-40.861412,...,-1.983973,4.916521,5.208999,-3.601717,-7.056323,-7.671651,1.058864,1.054138,3.543286,-0.483198
3,young,male,negative,-660.16850,-41.418015,47.211380,-25.102112,-4.699510,19.003640,11.318865,...,-10.161366,2.952621,-4.387035,8.939890,3.173635,2.953260,4.881428,8.365202,0.733032,4.489803
4,young,female,negative,-832.43820,97.745850,-13.485252,11.668908,2.601781,8.706614,-4.443015,...,12.727421,6.819866,-5.170315,-6.187433,-8.128935,1.426312,-1.929544,6.119107,1.827243,-0.272180


In [3]:
# determine number of positive and negative values for Covid Status column

data["Covid Status"].value_counts()

Covid Status
negative    709
positive    372
Name: count, dtype: int64

In [4]:
# since there are less positive values than negative values and the value types need to be the same for class 
# label balance, there needs to be a random sample of 372 negative values selected from the data

negatives = data[data["Covid Status"] == "negative"]
negatives_sample = negatives.sample(n = 372, random_state = 42)

In [5]:
# display the random sample of negative values

negatives_sample

,Age,Gender,Covid Status,1,2,3,4,5,6,7,...,31,32,33,34,35,36,37,38,39,40
470,young,male,negative,-543.15533,90.689514,28.970604,6.338865,9.591214,7.962400,8.999533,...,-5.214035,-0.327755,-1.228061,5.113654,-1.423935,2.840373,-1.566947,-3.431976,-3.701202,-2.503761
432,young,male,negative,-727.24084,164.063130,-19.572544,44.866737,-26.744781,22.994564,-29.662718,...,1.756853,0.415929,2.675990,0.054069,2.219139,-1.628018,-0.533565,-0.555858,1.751863,-1.296506
863,young,male,negative,-613.50780,17.359932,1.101616,49.072662,-48.691887,25.287312,-10.712454,...,-13.073709,0.920393,-6.196599,-0.008261,0.137898,-1.873153,-5.381612,-2.450135,-7.963347,-0.397056
549,young,male,negative,-710.35500,58.730850,22.247738,-13.321541,35.798714,23.302910,-3.378738,...,-4.017519,4.403957,-5.098961,-10.796859,-6.332717,-4.476096,-9.461203,7.662711,-10.089844,7.499491
424,young,male,negative,-1013.43750,82.807370,16.093830,9.230137,9.918889,17.411755,-1.945211,...,-1.060213,0.962077,-3.370188,-5.823066,0.351094,3.037789,-2.930956,-0.712987,-0.316106,-1.449122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,young,male,negative,-582.76960,194.736160,-26.117397,73.245820,-29.342022,32.692028,-27.051640,...,-2.787059,-13.819670,-18.762856,-12.226628,4.102395,0.530588,0.657076,0.077049,-3.855539,-1.892024
654,young,male,negative,-595.73627,120.411880,4.039909,7.232664,-7.251581,-18.158218,-14.684391,...,-4.092912,7.523797,-3.755001,8.982017,6.074190,-3.854938,-3.754892,-5.122377,-7.450123,4.354236
994,old,male,negative,-500.01987,180.660540,-127.236270,49.029655,-36.864563,21.681700,12.521856,...,-11.505336,-2.915974,8.138312,6.654276,-1.944061,-4.653092,4.477524,1.930779,0.670010,1.443905
308,young,male,negative,-552.64630,175.033450,-53.143684,87.825880,-29.637371,48.733900,-23.460133,...,-8.556097,-3.857112,1.875697,4.930209,-0.928397,-2.350164,3.783761,4.158632,-0.395517,2.957860


In [6]:
# extract the positive values from the data and combine the negatives_sample with the positive values into one 
# dataframe

positives = data[data["Covid Status"] == "positive"]
final_data = pd.concat([negatives_sample, positives])

In [7]:
# display the combined dataframe

final_data

,Age,Gender,Covid Status,1,2,3,4,5,6,7,...,31,32,33,34,35,36,37,38,39,40
470,young,male,negative,-543.15533,90.689514,28.970604,6.338865,9.591214,7.962400,8.999533,...,-5.214035,-0.327755,-1.228061,5.113654,-1.423935,2.840373,-1.566947,-3.431976,-3.701202,-2.503761
432,young,male,negative,-727.24084,164.063130,-19.572544,44.866737,-26.744781,22.994564,-29.662718,...,1.756853,0.415929,2.675990,0.054069,2.219139,-1.628018,-0.533565,-0.555858,1.751863,-1.296506
863,young,male,negative,-613.50780,17.359932,1.101616,49.072662,-48.691887,25.287312,-10.712454,...,-13.073709,0.920393,-6.196599,-0.008261,0.137898,-1.873153,-5.381612,-2.450135,-7.963347,-0.397056
549,young,male,negative,-710.35500,58.730850,22.247738,-13.321541,35.798714,23.302910,-3.378738,...,-4.017519,4.403957,-5.098961,-10.796859,-6.332717,-4.476096,-9.461203,7.662711,-10.089844,7.499491
424,young,male,negative,-1013.43750,82.807370,16.093830,9.230137,9.918889,17.411755,-1.945211,...,-1.060213,0.962077,-3.370188,-5.823066,0.351094,3.037789,-2.930956,-0.712987,-0.316106,-1.449122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1076,old,male,positive,-504.75726,55.995552,-4.536501,7.020762,-5.924695,2.774076,-7.127914,...,-5.155150,-7.399287,-7.925262,-5.619906,-5.644806,1.242933,-2.810986,-0.122058,-2.671896,0.472195
1077,young,male,positive,-723.34510,-5.390590,83.830670,-18.267603,23.992085,-17.434067,9.397023,...,-7.775133,16.577866,-2.542985,3.071694,-4.513388,-5.473712,0.011834,-0.797911,-1.411863,7.378503
1078,old,male,positive,-724.27490,96.186740,-65.017210,62.636177,-31.711601,48.414160,-7.423612,...,-4.680919,-4.848273,3.366529,-8.074186,-0.762944,1.346204,0.544383,-5.299528,0.331113,-0.893786
1079,young,male,positive,-525.03500,55.996940,-10.076442,20.016684,0.919657,-5.279638,10.089838,...,-1.043215,-2.091019,-5.030453,3.318240,-3.117649,-1.775076,-6.552867,-3.958253,-0.738340,1.371519


In [8]:
# use pandas to create dummy variables for the columns that have categorical value

one_hot_encoded_data = pd.get_dummies(final_data, columns = ["Age", "Gender", "Covid Status"])

In [9]:
# remove extra columns that were created from pandas dummy variables

one_hot_encoded_data = one_hot_encoded_data.loc[:, 
                                                ~one_hot_encoded_data.columns.isin(["Age_young", 
                                                                                    "Gender_female",  
                                                                                    "Covid Status_negative"])]
one_hot_encoded_data

,1,2,3,4,5,6,7,8,9,10,...,34,35,36,37,38,39,40,Age_old,Gender_male,Covid Status_positive
470,-543.15533,90.689514,28.970604,6.338865,9.591214,7.962400,8.999533,5.858377,9.145512,11.388906,...,5.113654,-1.423935,2.840373,-1.566947,-3.431976,-3.701202,-2.503761,False,True,False
432,-727.24084,164.063130,-19.572544,44.866737,-26.744781,22.994564,-29.662718,2.005984,0.186737,15.917499,...,0.054069,2.219139,-1.628018,-0.533565,-0.555858,1.751863,-1.296506,False,True,False
863,-613.50780,17.359932,1.101616,49.072662,-48.691887,25.287312,-10.712454,-0.336582,-12.418156,-4.193436,...,-0.008261,0.137898,-1.873153,-5.381612,-2.450135,-7.963347,-0.397056,False,True,False
549,-710.35500,58.730850,22.247738,-13.321541,35.798714,23.302910,-3.378738,10.243302,11.419808,2.328775,...,-10.796859,-6.332717,-4.476096,-9.461203,7.662711,-10.089844,7.499491,False,True,False
424,-1013.43750,82.807370,16.093830,9.230137,9.918889,17.411755,-1.945211,-8.004393,-8.872688,-14.920727,...,-5.823066,0.351094,3.037789,-2.930956,-0.712987,-0.316106,-1.449122,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1076,-504.75726,55.995552,-4.536501,7.020762,-5.924695,2.774076,-7.127914,-12.114542,-3.710113,-5.438988,...,-5.619906,-5.644806,1.242933,-2.810986,-0.122058,-2.671896,0.472195,True,True,True
1077,-723.34510,-5.390590,83.830670,-18.267603,23.992085,-17.434067,9.397023,-25.693218,5.133441,-13.971018,...,3.071694,-4.513388,-5.473712,0.011834,-0.797911,-1.411863,7.378503,False,True,True
1078,-724.27490,96.186740,-65.017210,62.636177,-31.711601,48.414160,-7.423612,6.581657,-14.663187,-3.385683,...,-8.074186,-0.762944,1.346204,0.544383,-5.299528,0.331113,-0.893786,True,True,True
1079,-525.03500,55.996940,-10.076442,20.016684,0.919657,-5.279638,10.089838,23.879969,2.035708,-17.630470,...,3.318240,-3.117649,-1.775076,-6.552867,-3.958253,-0.738340,1.371519,False,True,True


In [10]:
# for Age, True = Old and False = Young
# for Gender, True = Male and False = Female
# for Covid, True = Positive Test Result and False = Negative Test Result

one_hot_encoded_data.rename(columns = {"Age_old" : "Age", "Gender_male" : "Gender", 
                                       "Covid Status_positive" : "Covid"}, inplace = True)

In [11]:
one_hot_encoded_data

,1,2,3,4,5,6,7,8,9,10,...,34,35,36,37,38,39,40,Age,Gender,Covid
470,-543.15533,90.689514,28.970604,6.338865,9.591214,7.962400,8.999533,5.858377,9.145512,11.388906,...,5.113654,-1.423935,2.840373,-1.566947,-3.431976,-3.701202,-2.503761,False,True,False
432,-727.24084,164.063130,-19.572544,44.866737,-26.744781,22.994564,-29.662718,2.005984,0.186737,15.917499,...,0.054069,2.219139,-1.628018,-0.533565,-0.555858,1.751863,-1.296506,False,True,False
863,-613.50780,17.359932,1.101616,49.072662,-48.691887,25.287312,-10.712454,-0.336582,-12.418156,-4.193436,...,-0.008261,0.137898,-1.873153,-5.381612,-2.450135,-7.963347,-0.397056,False,True,False
549,-710.35500,58.730850,22.247738,-13.321541,35.798714,23.302910,-3.378738,10.243302,11.419808,2.328775,...,-10.796859,-6.332717,-4.476096,-9.461203,7.662711,-10.089844,7.499491,False,True,False
424,-1013.43750,82.807370,16.093830,9.230137,9.918889,17.411755,-1.945211,-8.004393,-8.872688,-14.920727,...,-5.823066,0.351094,3.037789,-2.930956,-0.712987,-0.316106,-1.449122,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1076,-504.75726,55.995552,-4.536501,7.020762,-5.924695,2.774076,-7.127914,-12.114542,-3.710113,-5.438988,...,-5.619906,-5.644806,1.242933,-2.810986,-0.122058,-2.671896,0.472195,True,True,True
1077,-723.34510,-5.390590,83.830670,-18.267603,23.992085,-17.434067,9.397023,-25.693218,5.133441,-13.971018,...,3.071694,-4.513388,-5.473712,0.011834,-0.797911,-1.411863,7.378503,False,True,True
1078,-724.27490,96.186740,-65.017210,62.636177,-31.711601,48.414160,-7.423612,6.581657,-14.663187,-3.385683,...,-8.074186,-0.762944,1.346204,0.544383,-5.299528,0.331113,-0.893786,True,True,True
1079,-525.03500,55.996940,-10.076442,20.016684,0.919657,-5.279638,10.089838,23.879969,2.035708,-17.630470,...,3.318240,-3.117649,-1.775076,-6.552867,-3.958253,-0.738340,1.371519,False,True,True


In [12]:
# X is the features, y is the target variable

X = one_hot_encoded_data.loc[:, one_hot_encoded_data.columns != "Covid"]

y = one_hot_encoded_data["Covid"]

In [26]:
# function to perform grid search cross validation and determine the optimal hyperparameters for the decision tree
# using 5 folds
# best_params are the best parameters
# best_score is the average performance 

def grid_search(X, y, cv):
    param_grid = {"criterion" : ["gini", "entropy"], "max_depth": np.arange(5, 15), 
                  "min_samples_split": [2, 3, 4, 5], "min_samples_leaf": [2, 3, 4, 5]}
    
    decision_tree = DecisionTreeClassifier()
    
    grid_search_cv = GridSearchCV(decision_tree, param_grid, cv = cv)
    grid_search_cv.fit(X, y)
    
    print("Best Parameters: ", grid_search_cv.best_params_)
    print("Best Score: ", grid_search_cv.best_score_)
    
    # return best estimator to use for the decision tree
    return grid_search_cv.best_estimator_

In [32]:
# accuracy score for decision tree model
# approximately 77% accurate
# approximately 575/744 samples classified correctly

best_estimator = grid_search(X, y, 5)
y_pred = best_estimator.predict(X)

print(accuracy_score(y, y_pred, normalize = True))
print(accuracy_score(y, y_pred, normalize = False))

Best Parameters:  {'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 4, 'min_samples_split': 3}
Best Score:  0.5563758389261746
0.7728494623655914
575


In [33]:
# lists to hold metric values for gender before mitigation algorithm for each of the 30 iterations
# for EQUALIZED ODDS, only looking at false negative rate, equalized odds ratio, and equalized odds difference

# No longer using the commented metrics for equalized odds
'''
eo_female_accuracy_before = []
eo_male_accuracy_before = []

eo_female_selection_rate_before = []
eo_male_selection_rate_before = []

eo_female_fpr_before = []
eo_male_fpr_before = []

eo_dpr_gender_before = []
eo_dpd_gender_before = []

'''

eo_female_fnr_before = []
eo_male_fnr_before = []

eo_eor_gender_before = []
eo_eod_gender_before = []

In [34]:
# lists to hold metric values for gender after mitigation algorithm for each of the 30 iterations
# for EQUALIZED ODDS, only looking at false negative rate, equalized odds ratio, and equalized odds difference

# No longer using the commented metrics for equalized odds
'''
eo_female_accuracy_after = []
eo_male_accuracy_after = []

eo_female_selection_rate_after = []
eo_male_selection_rate_after = []

eo_female_fpr_after = []
eo_male_fpr_after = []

eo_dpr_gender_after = []
eo_dpd_gender_after = []

'''

eo_female_fnr_after = []
eo_male_fnr_after = []

eo_eor_gender_after = []
eo_eod_gender_after = []

In [35]:
# lists to hold metric values for age before mitigation algorithm for each of the 30 iterations
# for EQUALIZED ODDS, only looking at false negative rate, equalized odds ratio, and equalized odds difference

# No longer using the commented metrics for equalized odds
'''
eo_young_accuracy_before = []
eo_old_accuracy_before = []

eo_young_selection_rate_before = []
eo_old_selection_rate_before = []

eo_young_fpr_before = []
eo_old_fpr_before = []

eo_dpr_age_before = []
eo_dpd_age_before = []

'''

eo_young_fnr_before = []
eo_old_fnr_before = []

eo_eor_age_before = []
eo_eod_age_before = []

In [36]:
# lists to hold metric values for age after mitigation algorithm for each of the 30 iterations
# for EQUALIZED ODDS, only looking at false negative rate, equalized odds ratio, and equalized odds difference

# No longer using the commented metrics for equalized odds
'''
eo_young_accuracy_after = []
eo_old_accuracy_after = []

eo_young_selection_rate_after = []
eo_old_selection_rate_after = []

eo_young_fpr_after = []
eo_old_fpr_after = []

eo_dpr_age_after = []
eo_dpd_age_after = []

'''

eo_young_fnr_after = []
eo_old_fnr_after = []

eo_eor_age_after = []
eo_eod_age_after = []

In [37]:
# run function 30 times
# get y_pred values 30 times
# get metric values 30 times before mitigation algorithm
# get metric values 30 times after mitigation algortihm
# metric values: false negative rate, equalized odds ratio, equalized odds difference
# mitigation algorithm: threshold optimizer (use equalized_odds for constraint)


for i in range(1, 31):
    # get best estimator from grid search cv
    best_estimator = grid_search(X, y, 5)
    
    # get y_pred values
    y_pred = best_estimator.predict(X)
    
    # metrics based on gender before mitigation
    # True = Male, False = Female
    print("ITERATION: ", i)
    
    metrics_gender = {"False Negative Rate" : false_negative_rate}


    metric_frame_gender = MetricFrame(metrics = metrics_gender, y_true = y, y_pred = y_pred, 
                                  sensitive_features = X["Gender"])

    # append to lists to hold metric values for gender before mitigation algorithm for each of the 30 iterations
    eo_female_fnr_before.append(metric_frame_gender.by_group["False Negative Rate"].iloc[0])
    eo_male_fnr_before.append(metric_frame_gender.by_group["False Negative Rate"].iloc[1])

    try:
        eo_eor_gender_before.append(fairlearn.metrics.equalized_odds_ratio(y_true = y, y_pred = y_pred, 
                                                     sensitive_features = X["Gender"], 
                                                     method = "between_groups"))
    except ZeroDivisionError:
        eo_eor_gender_before.append(0.0)
    
    eo_eod_gender_before.append(fairlearn.metrics.equalized_odds_difference(y_true = y, y_pred = y_pred, 
                                                     sensitive_features = X["Gender"], 
                                                     method = "between_groups"))
    
    # threshold optimizer with equalized odds for gender
    eo_threshold_optimizer_gender = ThresholdOptimizer(estimator = best_estimator, constraints = "equalized_odds", 
                                                predict_method = "predict_proba", 
                                                prefit = False)
    
    # fit the model and get y_pred values
    eo_threshold_optimizer_gender.fit(X, y, sensitive_features = X["Gender"])
    eo_y_pred_optimized_gender = eo_threshold_optimizer_gender.predict(X, sensitive_features = X["Gender"])
    
    # metrics based on gender after mitigation
    # True = Male, False = Female
    metric_frame_gender_optimized = MetricFrame(metrics = metrics_gender, y_true = y, 
                                                y_pred = eo_y_pred_optimized_gender, 
                                            sensitive_features = X["Gender"])

    # append to lists to hold metric values for gender after mitigation algorithm for each of the 30 iterations
    eo_female_fnr_after.append(metric_frame_gender_optimized.by_group["False Negative Rate"].iloc[0])
    eo_male_fnr_after.append(metric_frame_gender_optimized.by_group["False Negative Rate"].iloc[1])

    try:
        eo_eor_gender_after.append(fairlearn.metrics.equalized_odds_ratio(y_true = y, 
                                                                          y_pred = eo_y_pred_optimized_gender, 
                                                                          sensitive_features = X["Gender"], 
                                                                          method = "between_groups"))
    except ZeroDivisionError:
        eo_eor_gender_after.append(0.0)
    
    eo_eod_gender_after.append(fairlearn.metrics.equalized_odds_difference(y_true = y, 
                                                                           y_pred = eo_y_pred_optimized_gender, 
                                                     sensitive_features = X["Gender"], 
                                                     method = "between_groups"))

    # metrics based on age before mitigation
    # True = Old and False = Young
    metrics_age = {"False Negative Rate" : false_negative_rate}


    metric_frame_age = MetricFrame(metrics = metrics_age, y_true = y, y_pred = y_pred, 
                                   sensitive_features = X["Age"])

    # append lists to hold metric values for age before mitigation algorithm for each of the 30 iterations
    eo_young_fnr_before.append(metric_frame_age.by_group["False Negative Rate"].iloc[0])
    eo_old_fnr_before.append(metric_frame_age.by_group["False Negative Rate"].iloc[1])

    try:
        eo_eor_age_before.append(fairlearn.metrics.equalized_odds_ratio(y_true = y, y_pred = y_pred, 
                                                     sensitive_features = X["Age"], 
                                                     method = "between_groups"))
    except ZeroDivisionError:
        eo_eor_age_before.append(0.0)
        
    eo_eod_age_before.append(fairlearn.metrics.equalized_odds_difference(y_true = y, y_pred = y_pred, 
                                                     sensitive_features = X["Age"], 
                                                     method = "between_groups"))

    # threshold optimizer with equalized odds for age
    eo_threshold_optimizer_age = ThresholdOptimizer(estimator = best_estimator, constraints = "equalized_odds", 
                                                 predict_method = "predict_proba", 
                                                 prefit = False)
    
    # fit the model and get y_pred values
    eo_threshold_optimizer_age.fit(X, y, sensitive_features = X["Age"])
    eo_y_pred_optimized_age = eo_threshold_optimizer_age.predict(X, sensitive_features = X["Age"])
    
    # metrics based on age after mitigation
    # True = Old and False = Young
    metric_frame_age_optimized = MetricFrame(metrics = metrics_age, y_true = y, y_pred = eo_y_pred_optimized_age, 
                                         sensitive_features = X["Age"])

    # append lists to hold metric values for age after mitigation algorithm for each of the 30 iterations
    eo_young_fnr_after.append(metric_frame_age_optimized.by_group["False Negative Rate"].iloc[0])
    eo_old_fnr_after.append(metric_frame_age_optimized.by_group["False Negative Rate"].iloc[1])
    
    
    try:
        eo_eor_age_after.append(fairlearn.metrics.equalized_odds_ratio(y_true = y, 
                                                                       y_pred = eo_y_pred_optimized_age, 
                                                                       sensitive_features = X["Age"], 
                                                                       method = "between_groups"))
    except ZeroDivisionError:
        eo_eor_age_after.append(0.0)
    
    eo_eod_age_after.append(fairlearn.metrics.equalized_odds_difference(y_true = y, 
                                                                        y_pred = eo_y_pred_optimized_age, 
                                                                        sensitive_features = X["Age"], 
                                                                        method = "between_groups"))

Best Parameters:  {'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 3, 'min_samples_split': 4}
Best Score:  0.5604752403410121
ITERATION:  1
Best Parameters:  {'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 3, 'min_samples_split': 3}
Best Score:  0.5590876111010339
ITERATION:  2
Best Parameters:  {'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 2, 'min_samples_split': 4}
Best Score:  0.5563849083983312
ITERATION:  3
Best Parameters:  {'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 3, 'min_samples_split': 3}
Best Score:  0.5617721748594231
ITERATION:  4
Best Parameters:  {'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 3, 'min_samples_split': 5}
Best Score:  0.5604843098131689
ITERATION:  5
Best Parameters:  {'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 3, 'min_samples_split': 2}
Best Score:  0.5590876111010339
ITERATION:  6
Best Parameters:  {'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 3, 'min_samples

In [38]:
# convert results of metrics for gender to a dataframe

eo_results_gender = {
    "Female False Negative Rate Before": eo_female_fnr_before,
    "Male False Negative Rate Before": eo_male_fnr_before,
    "Female False Negative Rate After": eo_female_fnr_after,
    "Male False Negative Rate After": eo_male_fnr_after,
    "Equalized Odds Ratio Gender Before" : eo_eor_gender_before,
    "Equalized Odds Ratio Gender After" : eo_eor_gender_after,
    "Equalized Odds Difference Gender Before" : eo_eod_gender_before,
    "Equalized Odds Difference Gender After" : eo_eod_gender_after
}

eo_metric_results_gender = pd.DataFrame(eo_results_gender)
eo_metric_results_gender

,Female False Negative Rate Before,Male False Negative Rate Before,Female False Negative Rate After,Male False Negative Rate After,Equalized Odds Ratio Gender Before,Equalized Odds Ratio Gender After,Equalized Odds Difference Gender Before,Equalized Odds Difference Gender After
0,0.451852,0.582278,0.577778,0.556962,0.621156,0.894389,0.130427,0.020816
1,0.259259,0.341772,0.348148,0.337553,0.745387,0.974737,0.082513,0.010595
2,0.266667,0.345992,0.303704,0.337553,0.652214,0.869619,0.079325,0.033849
3,0.259259,0.337553,0.318519,0.320675,0.768681,0.996605,0.078293,0.002157
4,0.451852,0.582278,0.548148,0.556962,0.621156,0.894389,0.130427,0.008814
5,0.259259,0.341772,0.318519,0.333333,0.698801,0.878492,0.082513,0.016843
6,0.296296,0.413502,0.400000,0.405063,0.683272,0.813081,0.117206,0.022761
7,0.259259,0.333333,0.296296,0.324895,0.745387,0.843281,0.074074,0.028598
8,0.259259,0.337553,0.333333,0.341772,0.745387,0.922339,0.078293,0.009170
9,0.259259,0.341772,0.325926,0.329114,0.745387,0.995271,0.082513,0.003188


In [39]:
# convert average of each metric for gender to a dataframe

eo_averages_gender = pd.DataFrame(eo_metric_results_gender.mean()).T
eo_averages_gender

,Female False Negative Rate Before,Male False Negative Rate Before,Female False Negative Rate After,Male False Negative Rate After,Equalized Odds Ratio Gender Before,Equalized Odds Ratio Gender After,Equalized Odds Difference Gender Before,Equalized Odds Difference Gender After
0,0.318025,0.417159,0.399012,0.403797,0.684517,0.858202,0.099134,0.021628


In [40]:
# convert results of metrics for age to a dataframe

eo_results_age = {
    "Young False Negative Rate Before": eo_young_fnr_before,
    "Old False Negative Rate Before": eo_old_fnr_before,
    "Young False Negative Rate After": eo_young_fnr_after,
    "Old False Negative Rate After": eo_old_fnr_after,
    "Equalized Odds Ratio Age Before" : eo_eor_age_before,
    "Equalized Odds Ratio Age After" : eo_eor_age_after,
    "Equalized Odds Difference Age Before" : eo_eod_age_before,
    "Equalized Odds Difference Age After" : eo_eod_age_after
}

eo_metric_results_age = pd.DataFrame(eo_results_age)
eo_metric_results_age

,Young False Negative Rate Before,Old False Negative Rate Before,Young False Negative Rate After,Old False Negative Rate After,Equalized Odds Ratio Age Before,Equalized Odds Ratio Age After,Equalized Odds Difference Age Before,Equalized Odds Difference Age After
0,0.738776,0.141732,0.077551,0.062992,0.304362,0.876405,0.597043,0.087640
1,0.461224,0.023622,0.440816,0.440945,0.523967,0.352876,0.437602,0.100029
2,0.461224,0.039370,0.440816,0.472441,0.134038,0.594863,0.421854,0.103126
3,0.461224,0.015748,0.440816,0.417323,0.547396,0.944620,0.445476,0.023493
4,0.738776,0.141732,0.077551,0.102362,0.304362,0.919089,0.597043,0.054431
5,0.461224,0.023622,0.440816,0.480315,0.261983,0.705751,0.437602,0.045483
6,0.534694,0.055118,0.534694,0.496063,0.235250,0.588126,0.479576,0.063665
7,0.461224,0.007874,0.440816,0.488189,0.523967,0.915282,0.453350,0.047373
8,0.461224,0.015748,0.440816,0.488189,0.523967,0.772871,0.445476,0.047373
9,0.461224,0.023622,0.440816,0.377953,0.523967,0.772871,0.437602,0.062864


In [41]:
# convert average of each metric for age to a dataframe

eo_averages_age = pd.DataFrame(eo_metric_results_age.mean()).T
eo_averages_age

,Young False Negative Rate Before,Old False Negative Rate Before,Young False Negative Rate After,Old False Negative Rate After,Equalized Odds Ratio Age Before,Equalized Odds Ratio Age After,Equalized Odds Difference Age Before,Equalized Odds Difference Age After
0,0.549388,0.056693,0.337279,0.33622,0.357271,0.820257,0.492695,0.052746


In [42]:
# save metric_results_gender, averages_gender, metric_results_age, and averages_age dataframes as csv files

eo_metric_results_gender.to_csv("equalized_odds_metric_results_by_gender_coswara_data.csv", index = False)
eo_averages_gender.to_csv("equalized_odds_averages_for_gender_coswara_data.csv", index = False)

eo_metric_results_age.to_csv("equalized_odds_metric_results_by_age_coswara_data.csv", index = False)
eo_averages_age.to_csv("equalized_odds_averages_for_age_coswara_data.csv", index = False)